In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import uniform, randint
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importing data
df = pd.read_csv("../data/finaldataset.csv")

In [4]:
#same preprocessing steps as Darts models

#Remove countries with very few observations
df = df[df["country_d"] != "Taiwan"]
df = df[df["country_d"] != "North Korea"]
df = df[df["country_d"] != "Liechtenstein"]
df = df[df["country_d"] != "Monaco"]
df = df[df["country_d"] != "Botswana"]
df = df[df["country_d"] != "Lesotho"]
df = df[df["country_d"] != "Luxembourg"]
df = df[df["country_d"] != "Yugoslavia"]
df = df[df["country_d"] != "Sao Tome and Principe"]
df = df[df["country_d"] != "Iraq"]
df = df[df["country_d"] != "Sudan"]
df = df[df["country_d"] != "Equatorial Guinea"]

In [5]:
#Obtaining bilateral trade value:
#Use Singapore-reported export value by default, if NaN, use trade partner reported import value
#combined trade series ensure broader coverage
#method adapted from CEPII Trade and Production database (TradeProd)
#https://www.cepii.fr/DATA_DOWNLOAD/tradeprod/V202401/CEPII_TradeProd_V202401.pdf

df["all_exports_new"] = np.where(
    df["tradeflow_comtrade_o"].notna(),
    df["tradeflow_comtrade_o"],
    np.where(
        df["tradeflow_comtrade_d"].notna(),
        df["tradeflow_comtrade_d"],
        np.nan
    )
)

In [6]:
df["log_all_exports_new"] = np.log1p(df["all_exports_new"])
df["log_gdpcap_d"] = np.log1p(df["gdpcap_d"])
df["log_pop_d"] = np.log1p(df["pop_d"])
df["log_distcap"] = np.log1p(df["distcap"])

In [7]:
# Sorting by (country_d, year)
df_2 = df.sort_values(by=['country_d', 'year'])

# Set lag variables
df_2['lag_log_pop_d'] = df_2.groupby('country_d')['log_pop_d'].shift(1)
df_2['lag_log_gdpcap_d'] = df_2.groupby('country_d')['log_gdpcap_d'].shift(1)
df_2['lag_geodistance'] = df_2.groupby('country_d')['geodistance'].shift(1)

df_2 = df_2[df_2.groupby('country_d')['year'].diff() == 1]


In [8]:
df_2[['lag_geodistance', 'log_distcap', 'lag_log_gdpcap_d', 'lag_log_pop_d']].corrwith(df_2['log_all_exports_new'])

# signs of each variable's correlation with target variable are expected

,0
lag_geodistance,-0.139940
log_distcap,-0.414728
lag_log_gdpcap_d,0.460146
lag_log_pop_d,0.487271


In [9]:
full_years = pd.date_range(start="1989-12-31", end="2020-12-31", freq="YE")
full_years_df = pd.DataFrame({"year": full_years})

country_frames = []

# Loop through each country
for country in df_2['country_d'].unique():

    # Filter for current country
    df_country = df_2[df_2['country_d'] == country].copy()

    # Ensure all years are present
    full_years_country = full_years_df.copy()
    full_years_country['country_d'] = country

    df_country["year"] = pd.to_datetime(df_country["year"], format="%Y") + pd.offsets.YearEnd(0)

    df_country = pd.merge(full_years_country, df_country, on=["year", "country_d"], how="left")

    # Drop duplicate years (in case of prior merge artifacts)
    df_country = df_country.drop_duplicates(subset=["year"], keep="first")

    vars_to_fill = ['log_all_exports_new', 'lag_geodistance', 'log_distcap', 'lag_log_gdpcap_d', 'lag_log_pop_d']

    before_2009 = df_country["year"] <= pd.Timestamp("2008-12-31")
    df_before_2009 = df_country[before_2009].copy()

    # Interpolate pre-2009
    for var in vars_to_fill:
        df_country.loc[before_2009, var] = (
            df_country.loc[before_2009, var]
            .interpolate(method="linear", limit_direction="both")
        )

    # Handle 2009 special case
    mask_2009 = df_country["year"] == pd.Timestamp("2009-12-31")
    for var in vars_to_fill:
        if df_country.loc[mask_2009, var].isna().any():
            last_val = df_before_2009[var].dropna().iloc[-1]
            df_country.loc[mask_2009, var] = last_val


    # Forward-fill for 2009 onwards
    from_2009 = df_country["year"] >= pd.Timestamp("2009-12-31")
    df_country.loc[from_2009, vars_to_fill] = (
        df_country.loc[from_2009, vars_to_fill].ffill()
    )

    country_frames.append(df_country)


df_2 = pd.concat(country_frames).reset_index(drop=True)


In [10]:
#expanding window cv function

def expanding_window_cv_countrywise(model, X, y, countries, year_col='year', min_year=1989, max_year=2020, min_train_years=20):
    min_year = pd.Timestamp(f"{min_year}-12-31")
    max_year = pd.Timestamp(f"{max_year}-12-31")

    country_metrics = []
    all_predictions = []

    for country in countries:
        # Filter by country
        country_mask = X['country_d'] == country
        X_country = X.loc[country_mask].copy()
        y_country = y.loc[country_mask].copy()

        # Filter by years
        year_mask = (X_country[year_col] >= min_year) & (X_country[year_col] <= max_year)
        X_country = X_country.loc[year_mask]
        y_country = y_country.loc[year_mask]

        unique_years = sorted(X_country[year_col].unique())
        if len(unique_years) <= min_train_years:
            continue

        predictions = pd.Series(index=y_country.index, dtype=float)
        y_true_total = []
        y_pred_total = []

        for i in range(min_train_years, len(unique_years) - 1):
            train_years = unique_years[:i]
            test_year = unique_years[i]

            train_mask = X_country[year_col].isin(train_years)
            test_mask = X_country[year_col] == test_year

            X_train = X_country.loc[train_mask].drop(columns=[year_col, 'country_d'])
            X_test = X_country.loc[test_mask].drop(columns=[year_col, 'country_d'])
            y_train = y_country.loc[train_mask]
            y_test = y_country.loc[test_mask]

            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Store actual and predicted
            y_true_total.extend(y_test.values.flatten())
            y_pred_total.extend(y_pred.flatten())

            predictions.loc[y_test.index] = y_pred.flatten()

        if y_true_total:
            # Compute single metric per country from all y_true vs y_pred
            mse = mean_squared_error(y_true_total, y_pred_total)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_true_total, y_pred_total)
            mape = mean_absolute_percentage_error(y_true_total, y_pred_total)

            country_metrics.append({
                'country': country,
                'mse': mse,
                'rmse': rmse,
                'mae': mae,
                'mape': mape,
                'n_preds': len(y_true_total)
            })

            all_predictions.append(predictions)

    # Combine all predictions
    final_predictions = pd.concat(all_predictions).sort_index()

    # Average metrics across countries
    if country_metrics:
        avg_metrics = {
            'avg_mse': np.mean([m['mse'] for m in country_metrics]),
            'avg_rmse': np.mean([m['rmse'] for m in country_metrics]),
            'avg_mae': np.mean([m['mae'] for m in country_metrics]),
            'avg_mape': np.mean([m['mape'] for m in country_metrics]),
            'countries': len(country_metrics)
        }
    else:
        avg_metrics = {}

    return final_predictions, avg_metrics, country_metrics


In [11]:
xgboostX = df_2[['country_d', 'log_distcap', 'lag_geodistance', 'lag_log_pop_d', 'lag_log_gdpcap_d', 'year']].copy()
xgboostY = df_2[['log_all_exports_new']]

# Run country-wise cross-validation
countries = df_2['country_d'].unique()

xgboostModel = xgb.XGBRegressor(
    n_estimators = 100,
    max_depth = 4,
    learning_rate = 0.05,
    subsample = 0.8,
    colsample_bytree = 0.8,
    random_state = 222
)

xgboostPredictions, xgboostAvgMetrics, xgboostCountryMetrics = expanding_window_cv_countrywise(
    xgboostModel, xgboostX, xgboostY, countries
)

# Print summary metrics
print("Average Metrics Across Countries:")
print(xgboostAvgMetrics)

Average Metrics Across Countries:
{'avg_mse': np.float64(0.576150598813245), 'avg_rmse': np.float64(0.613385048135207), 'avg_mae': np.float64(0.4792252223754813), 'avg_mape': np.float64(0.02942200276850364), 'countries': 161}


In [12]:
from sklearn.ensemble import RandomForestRegressor

rfX = df_2[['country_d', 'log_distcap', 'lag_geodistance', 'lag_log_pop_d', 'lag_log_gdpcap_d', 'year']].copy()
rfY = df_2[['log_all_exports_new']]

# Run country-wise cross-validation
countries = df_2['country_d'].unique()

rfModel = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=222,
    n_jobs=-1
)

rfPredictions, rfAvgMetrics, rfCountryMetrics = expanding_window_cv_countrywise(
    rfModel, rfX, rfY, countries
)

# Print summary metrics
print("Average Metrics Across Countries:")
print(rfAvgMetrics)


Average Metrics Across Countries:
{'avg_mse': np.float64(0.591814404321084), 'avg_rmse': np.float64(0.6157920805512621), 'avg_mae': np.float64(0.4863432130973088), 'avg_mape': np.float64(0.029994500920421763), 'countries': 161}
